In [11]:
from itertools import combinations
import numpy as np
import tensorflow as tf

In [13]:
for i in combinations(a, 2):
  print(i)

(1, 2)
(1, 3)
(2, 3)


In [5]:
a = np.array([1,2,3])
print(a[:2])

[1 2]


In [17]:
s = [(1,2),(3,4),(-1,2)]
if (3,2) in s:
  print('yes')
  
s += ((5,7),)
print(s)

[(1, 2), (3, 4), (-1, 2), (5, 7)]


In [19]:
print(s.sort(key=lambda x:x[0]))
print(s)

None
[(-1, 2), (1, 2), (3, 4), (5, 7)]


In [28]:
X = [[(i*j, -i) for i in range(3)] for j in range(5)]
X.sort(key=lambda x: x[0][0])
print(X)

[[(0, 0), (0, -1), (0, -2)], [(0, 0), (1, -1), (2, -2)], [(0, 0), (2, -1), (4, -2)], [(0, 0), (3, -1), (6, -2)], [(0, 0), (4, -1), (8, -2)]]


In [16]:
k = [(1,2),(4,5),(2,7)]

a = k
a = [(1,2)]
print(k)

[(1, 2), (4, 5), (2, 7)]


In [20]:
a = {'a': 1, 'b': 2}
for i in a:
  print(i)

a
b


In [22]:
b =[2,3,4,1,3]
for i, val in enumerate(a):
  print(i, val)

0 a
1 b


In [2]:
l = (1,2)
l[0] = 7
print(l)

TypeError: 'tuple' object does not support item assignment

In [ ]:
def cornersHeuristic(state, problem, prev_state):
    """
    A heuristic for the CornersProblem that you defined.

      state:   The current search state
               (a data structure you chose in your search problem)

      problem: The CornersProblem instance for this layout.

    This function should always return a number that is a lower bound on the
    shortest path from the state to a goal of the problem; i.e.  it should be
    admissible (as well as consistent).
    """
    corners = problem.corners # These are the corner coordinates
    walls = problem.walls # These are the walls of the maze, as a Grid (game.py)

    "*** YOUR CODE HERE ***"

    if not prev_state:
        return min([manhattanDist(state[0], goal) for goal in corners])
    prev_pos, prev_corners_reached = prev_state
    # if pos in corners:
    #     if corners_reached[corners.index(pos)] == True:
    #         return 0
    
    prev_state_distances = [(manhattanDist(prev_pos, goal), corners.index(goal)) for goal in corners if not prev_corners_reached[corners.index(goal)]]

    prev_state_distances.sort(key=lambda x: x[0])
    
    # if not successors:
    #     return state_distances[-1][0]
    pos, corners_reached = state
    state_distances = [(manhattanDist(pos, corners[x[1]]), x[1]) for x in prev_state_distances]
    
    for idx in range(len(state_distances)):
        dist = state_distances[idx][0]
        if dist > prev_state_distances[idx][0] and dist > 0:
            return dist

    return max(state_distances, key=lambda x: x[0])[0]
        # if prev_state_distances
    
    # succ_distances = [[(manhattanDist(s[0][0], goal), corners.index(goal)) for goal in corners if not corners_reached[corners.index(goal)]] for s in successors]
    # print(succ_distances)
    # print('\n')
    # if not succ_distances[0]:
        
    # succ_distances.sort(key=lambda x: x[0][0])
    
    # order_heuristic = [[] for _ in range(len(state_distances))]
    
    # for idx in range(len(state_distances)):
    #     for s in succ_distances:
    #         if s[idx][1] == state_distances[idx][1] and s[idx][0] < state_distances[idx][0]:
    #             order_heuristic[idx].append(s[idx][0])
    #     if order_heuristic[idx]:
    #         return min(order_heuristic[idx])

    # return state_distances[-1][0]

### This results in even more nodes being expanded. Let's take a step back and make sure both of us are on the same page in terms of understanding the purpose of my heuristic and that it will really help in expanding less number of nodes. So, in a sense by checking how close the successor gets to an already closest goal corner, we are providing a good direction in terms of being able to reach an already close and 'reachable' goal. We have to make sure that we are not naive to the walls of the maze. I already check for legal successors when providing them and before pushing them into the priority queue. But, if a successor is unable to get closer to an already closest goal state, then we have to assume that there must be some other sibling successor that does that. Although, if none of the successors go close to an already closest goal state then that means that there is a wall block out a 'what could have been' the direct path to that goal. I want to encode this information as well. That is if none get closer to the closest goal, then we must proceed towards a corner which is the second closest and choose a successor which is the closest to this second best goal. However, at a time while evaluating the heuristic of a successor, we only have access to its previous state. Using, these details and intuition, I want you to devise this heuristic which is also consistent. You may also refer to the code that I provided earlier (which was consistent). But, make sure to be logically accurate.

In [ ]:
def aStarSearch(problem, heuristic=nullHeuristic):
    """Search the node that has the lowest combined cost and heuristic first."""
    "*** YOUR CODE HERE ***"
    
    L = PriorityQueue()
    
    start_node = Node(problem.getStartState(), None, None, 0, 0)

    # L.push(start_node, start_node.cost + heuristic(start_node.state, problem, []))
    L.push(start_node, start_node.cost + heuristic(start_node.state, problem))
    visited = set()
    v = set()
    H = set()
    while not L.isEmpty():
        current_node = L.pop()
        current_state = current_node.state
        if problem.isGoalState(current_state):
            # printHeatMap(H, problem)
            return current_node.getPath()
        # h = []
        if current_state[:2] not in visited:
            if current_state[0] not in v:
                H.add((current_state[0], current_node.h))
                v.add(current_state[0])
                
            visited.add(current_state[:2])
            successors = problem.getSuccessors(current_state)
            # if not successors:
            #     print('HI')
            # print(len(H))
            # if len(H) == 194:
            #     printHeatMap(H, problem)
            # np.savetxt('file3.txt', np.array(H))
            for s in successors:
                # if s[:2] not in visited:
                    temp = heuristic(s[0], problem)
                    # h.append(temp)
                    L.push(Node(s[0], s[1], current_node, current_node.cost + s[2], current_node.cost + s[2] + temp), current_node.cost + s[2] + temp)
        # H.append(h)
    
    return []

In [ ]:
        for action in [Directions.NORTH, Directions.SOUTH, Directions.EAST, Directions.WEST]:
            pos, corners_reached, _ = state
            x, y = pos
            dx, dy = Actions.directionToVector(action)
            nextx, nexty = int(x + dx), int(y + dy)
            next_pos = (nextx, nexty)

            if not self.walls[nextx][nexty]:
                prev_corners_reached = corners_reached
                if next_pos in self.corners:
                    corners_reached = list(corners_reached)
                    corners_reached[self.corners.index(next_pos)] = True
                    # self.visited.add(next_pos)
                successors.append( ((next_pos, tuple(corners_reached), (pos, prev_corners_reached)), action, 1) )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from Node import Node
from scipy.interpolate import griddata

def printHeatMap(H, problem):

    coordinates, h_values = zip(*H)
    x_coords, y_coords = zip(*coordinates)
    x_coords = np.array(x_coords)
    y_coords = np.array(y_coords)
    h_values = np.array(h_values)

    walls_list = list(problem.gameState.getWalls())
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    x_grid = np.linspace(min(x_coords), max(x_coords), 100)
    y_grid = np.linspace(min(y_coords), max(y_coords), 100)
    x_grid, y_grid = np.meshgrid(x_grid, y_grid)

    z_values = griddata((x_coords, y_coords), h_values, (x_grid, y_grid), method='cubic')

    ax.plot_surface(x_grid, y_grid, z_values, cmap='cool')

    
    walls = np.array(walls_list)
    W = [[(i, j) for j in range(walls.shape[1]) if walls[i,j] == True] for i in range(walls.shape[0])]
    for coordinates in W:
        if coordinates:
            x, y = zip(*coordinates)
            plt.scatter(x, y,s=150,c='blue')

    ax.set_xlim(0, 30)  # adjust the values as needed
    ax.set_ylim(0, 30)  # adjust the values as needed
    ax.set_title('Heuristic Evaluation 3D Plot with walls')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_zlabel('Heuristic Value')

    plt.show()


In [ ]:
    def getStartState(self):
        """
        Returns the start state (in your state space, not the full Pacman state
        space)
        """
        "*** YOUR CODE HERE ***"
        # res = (self.startingPosition, (False, False, False, False), ((), (False, False, False, False)))
        res = (self.startingPosition, (False, False, False, False), (1))
        return res

    def isGoalState(self, state):
        """
        Returns whether this search state is a goal state of the problem.
        """
        
        for reached in state[1]:
            if not reached:
                return False
        return True

        
    def getSuccessors(self, state):
        """
        Returns successor states, the actions they require, and a cost of 1.

        As noted in search.py:
            For a given state, this should return a list of triples, (successor,
            action, stepCost), where 'successor' is a successor to the current
            state, 'action' is the action required to get there, and 'stepCost'
            is the incremental cost of expanding to that successor
        """
        successors = []
        succ_distances = []
            
        for action in [Directions.NORTH, Directions.SOUTH, Directions.EAST, Directions.WEST]:
            pos, corners_reached, rank = state
            state_distances = [(manhattanDist(pos, goal), self.corners.index(goal)) for goal in self.corners if not corners_reached[self.corners.index(goal)]]
            state_distances.sort(key=lambda x: x[0])
            x, y = pos
            dx, dy = Actions.directionToVector(action)
            nextx, nexty = int(x + dx), int(y + dy)
            next_pos = (nextx, nexty)

            if not self.walls[nextx][nexty]:     
                if next_pos in self.corners:
                    corners_reached = list(corners_reached)
                    corners_reached[self.corners.index(next_pos)] = True
                s_distances = [(manhattanDist(next_pos, self.corners[x[1]]), x[1]) for x in state_distances]
                succ_distances.append(s_distances)
                successors.append( ((next_pos, tuple(corners_reached), max(s_distances, key=lambda x: x[0])[0]), action, 1) )
        
        ranks = [[] for _ in range(len(state_distances))]

        for i in range(len(successors)):
            for idx in range(len(state_distances)):
                d = succ_distances[i][idx][0]
                if d < state_distances[idx][0] and d > 0:
                    ranks[idx].append((i, d))

        for i, rank in enumerate(ranks):
            if not rank:
                continue
            else:
                min_rank = min(rank, key=lambda x: x[1])
                updated_successor = list(successors[min_rank[0]])
                updated_successor[0] = (updated_successor[0][0], updated_successor[0][1], min_rank[1])
                successors[min_rank[0]] = tuple(updated_successor)

        self._expanded += 1  # DO NOT CHANGE
        
        return successors

In [ ]:
foodList = foodGrid.asList()

    if not foodList:
        return 0

    # vertices = [f for f in foodList if f != position]
    vertices = [position] + foodList
    # closest_food = min(((v, util.manhattanDistance(position, v)) for v in vertices), key=lambda x: x[0])
    # closest_item, closest_cost = closest_food
    # if closest_cost == 0:
        # print('ok')
    # print(len(vertices))
    vertices_cost = {vertex: float('inf') for vertex in vertices}
    vertices_cost[closest_item] = 0
    visited = set()
    mst_cost = 0
    while len(visited) < len(vertices):
        current_vertex = min((v for v in vertices if v not in visited), key=lambda v: vertices_cost[v])
        # if position == problem.start[0] and mst_cost == 0 and len(visited) == 0:
        #     print(current_vertex)
        visited.add(current_vertex)
        mst_cost += vertices_cost[current_vertex]

        for vertex in vertices:
            dist = util.manhattanDistance(current_vertex, vertex)
            if vertex not in visited and dist < vertices_cost[vertex]:
                vertices_cost[vertex] = dist

    # if len(foodList) == 2:
    #     print(f'MST cost: {mst_cost} for {foodList}')
    # print('-------')
    return mst_cost

In [ ]:
    def updateAdjMatrix(self, currentFood):
        foodMap = self.heuristicInfo['foodMap']
        currentAdjMatrix = self.heuristicInfo['adjMatrix'].copy()
        for i, f in enumerate(foodMap):
            if f not in currentFood:
                currentAdjMatrix[i, :] = 0
                currentAdjMatrix[:, i] = 0
        return currentAdjMatrix

 
    def computeMSTCost(self, adj_matrix):

        # num_vertices = len(adj_matrix)
        # selected = [False] * num_vertices
        # key_values = [float('inf')] * num_vertices
        # key_values[0] = 0
        # mst_weight = 0

        # for _ in range(num_vertices):
        #     # Find the vertex with the minimum key value not yet added to MST
        #     min_value = float('inf')
        #     for v in range(num_vertices):
        #         if not selected[v] and key_values[v] < min_value:
        #             min_value = key_values[v]
        #             u = v

        #     # Add the selected vertex to MST
        #     selected[u] = True
        #     mst_weight += min_value

        #     # Update the key values of adjacent vertices
        #     for v in range(num_vertices):
        #         if adj_matrix[u][v] > 0 and not selected[v] and adj_matrix[u][v] < key_values[v]:
        #             key_values[v] = adj_matrix[u][v]

    # if len(foodList) == 2:
    #     print(f'MST cost: {mst_cost} for {foodList}')
    # print('-------')
    # return mst_cost

        # Calculate MST weight using NetworkX
        g = nx.from_numpy_array(adj_matrix)
        mst = nx.minimum_spanning_tree(g)
        m_prime = sum(e[2]['weight'] for e in mst.edges(data=True))
        # m_prim = self.primMST(adj_matrix, len(adj_matrix))

        # Check if the computed MST weight matches the MST weight computed using your algorithm
        # assert m_prime == m_prim, f'm_prime: {m_prime}, m_prim: {m_prim}'



        return m_prime